# 5X Finder Feature Validation
Phase 1 - 간소화된 EDA

**목표:**
1. 데이터 수집 테스트
2. Feature 계산 검증
3. 결측치 비율 확인
4. 분포 시각화
5. 이상치 탐지

## 1. 환경 설정

In [2]:
import sys
import os

# backend 경로 추가 (쿠카 환경에 맞게 설정)
backend_path = '/Users/kuka/LongArc/backend'

if backend_path not in sys.path:
    sys.path.insert(0, backend_path)

print(f"Backend 경로: {backend_path}")
print(f"경로 존재: {os.path.exists(backend_path)}")

Backend 경로: /Users/kuka/LongArc/backend
경로 존재: True


In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("✅ 환경 설정 완료")

✅ 환경 설정 완료


## 2. 데이터 수집 테스트

In [4]:
from data.collector import DataCollector
from data.universe import Universe
from features.calculator import FeatureCalculator 

# 유니버스 로드
universe = Universe()
collector = DataCollector()
calculator = FeatureCalculator()

# 테스트용 종목 (다양한 섹터)
TEST_TICKERS = [
    'AAPL',   # Tech - 하드웨어
    'MSFT',   # Tech - 소프트웨어
    'NVDA',   # Tech - 반도체
    'GOOGL',  # Communication - 광고
    'AMZN',   # Consumer - 이커머스
    'TSLA',   # Consumer - EV
    'UNH',    # Healthcare
    'JPM',    # Financials
    'XOM',    # Energy
    'CAT',    # Industrials
]

print(f"테스트 종목: {len(TEST_TICKERS)}개")

테스트 종목: 10개


In [5]:
# 데이터 수집 (이미 수집된 경우 로드)
collected_data = {}

for ticker in tqdm(TEST_TICKERS, desc="데이터 수집"):
    # 가격 데이터
    prices = collector.load_price_data(ticker)
    if prices is None:
        prices = collector.collect_price_history(ticker, years=10)
    
    # 재무제표
    financials = collector.load_financials(ticker)
    if financials is None:
        financials = collector.collect_financials(ticker)
    
    if prices is not None and financials is not None:
        collected_data[ticker] = {
            'prices': prices,
            'financials': financials
        }

print(f"\n✅ 수집 완료: {len(collected_data)}/{len(TEST_TICKERS)} 종목")

데이터 수집: 100%|██████████| 10/10 [00:00<00:00, 10.78it/s]


✅ 수집 완료: 10/10 종목


In [6]:
# 수집된 데이터 구조 확인
sample_ticker = list(collected_data.keys())[0]
sample = collected_data[sample_ticker]

print(f"=== {sample_ticker} 데이터 구조 ===")
print()
print("📈 가격 데이터:")
print(f"   Shape: {sample['prices'].shape}")
print(f"   기간: {sample['prices']['Date'].min()} ~ {sample['prices']['Date'].max()}")
print(f"   컬럼: {list(sample['prices'].columns)}")

print()
print("📊 재무제표:")
for key, df in sample['financials'].items():
    if isinstance(df, pd.DataFrame):
        print(f"   {key}: {df.shape}")
    elif isinstance(df, dict):
        print(f"   {key}: {len(df)} items")

=== AAPL 데이터 구조 ===

📈 가격 데이터:
   Shape: (5027, 7)
   기간: 2006-01-03 00:00:00 ~ 2025-12-24 00:00:00
   컬럼: ['Date', 'Open', 'High', 'Low', 'Close', 'AdjClose', 'Volume']

📊 재무제표:
   balance_sheet_quarterly: (5, 66)
   balance_sheet_annual: (5, 70)
   income_stmt_annual: (5, 40)
   cash_flow_annual: (5, 54)
   info: 182 items
   income_stmt_quarterly: (5, 34)
   cash_flow_quarterly: (7, 47)


## 3. Feature 계산 테스트

In [7]:
import yfinance as yf

# Market Cap 확인
for ticker in TEST_TICKERS:
    try:
        info = yf.Ticker(ticker).info
        market_cap = info.get('marketCap', 'N/A')
        print(f"{ticker}: {market_cap:,}" if isinstance(market_cap, int) else f"{ticker}: {market_cap}")
    except:
        print(f"{ticker}: Error")

AAPL: 4,057,362,333,696
MSFT: 3,625,229,484,032
NVDA: 4,638,834,163,712
GOOGL: 3,797,271,445,504
AMZN: 2,485,688,991,744
TSLA: 1,580,396,052,480
UNH: 300,584,402,944
JPM: 901,671,813,120
XOM: 507,795,374,080
CAT: 273,123,213,312


In [8]:
# Feature 계산 후 펼치기
all_features = []

for ticker, data in tqdm(collected_data.items(), desc="Feature 계산"):
    financials = data['financials']
    
    income_stmt = financials.get('income_stmt_annual', pd.DataFrame())
    balance_sheet = financials.get('balance_sheet_annual', pd.DataFrame())
    cash_flow = financials.get('cash_flow_annual', pd.DataFrame())
    price_history = data['prices']
    
    try:
        market_cap = yf.Ticker(ticker).info.get('marketCap', None)
    except:
        market_cap = None
    
    result = calculator.calculate_all(
        ticker=ticker,
        income_stmt=income_stmt,
        balance_sheet=balance_sheet,
        cash_flow=cash_flow,
        price_history=price_history,
        market_cap=market_cap
    )
    
    if result and 'all_features' in result:
        # all_features dict를 펼치기
        features = result['all_features']
        features['ticker'] = ticker
        all_features.append(features)

features_df = pd.DataFrame(all_features).set_index('ticker')
print(f"\\n✅ Feature 계산 완료")
print(f"   종목 수: {len(features_df)}")
print(f"   Feature 수: {len(features_df.columns)}")

Feature 계산: 100%|██████████| 10/10 [00:02<00:00,  3.39it/s]

\n✅ Feature 계산 완료
   종목 수: 10
   Feature 수: 42


In [9]:
# Feature 목록 확인
print("=== 계산된 Feature 목록 ===")
print()
for i, col in enumerate(features_df.columns, 1):
    print(f"{i:2d}. {col}")

=== 계산된 Feature 목록 ===

 1. revenue_cagr_3y
 2. revenue_cagr_5y
 3. gross_profit_cagr_3y
 4. operating_income_cagr_3y
 5. net_income_cagr_3y
 6. gross_margin
 7. operating_margin
 8. net_margin
 9. fcf_margin
10. operating_margin_trend
11. roe
12. roa
13. roic
14. rnd_to_revenue
15. rnd_growth_rate
16. capex_to_revenue
17. capex_to_depreciation
18. reinvestment_rate
19. total_investment_ratio
20. debt_to_equity
21. debt_to_assets
22. interest_coverage
23. current_ratio
24. fcf_consistency
25. fcf_positive_years
26. earnings_quality
27. ps_ratio
28. pe_ratio
29. pb_ratio
30. peg_ratio
31. ev_to_revenue
32. ev_to_ebitda
33. fcf_yield
34. price_momentum_12m
35. price_momentum_6m
36. price_momentum_3m
37. price_momentum_1m
38. volatility_1y
39. volatility_3m
40. pct_from_52w_high
41. price_to_sma_50
42. price_to_sma_200


## 4. 결측치 분석

In [10]:
# 결측치 비율 계산
missing_ratio = features_df.isnull().sum() / len(features_df) * 100
missing_df = pd.DataFrame({
    'feature': missing_ratio.index,
    'missing_pct': missing_ratio.values
}).sort_values('missing_pct', ascending=False)

print("=== 결측치 비율 (상위 15개) ===")
print()
print(missing_df.head(15).to_string(index=False))

=== 결측치 비율 (상위 15개) ===

                 feature  missing_pct
         revenue_cagr_5y        100.0
          rnd_to_revenue         40.0
         rnd_growth_rate         40.0
   capex_to_depreciation         20.0
  operating_margin_trend         10.0
        operating_margin         10.0
              fcf_margin         10.0
operating_income_cagr_3y         10.0
            gross_margin         10.0
    gross_profit_cagr_3y         10.0
         fcf_consistency         10.0
           current_ratio         10.0
            ev_to_ebitda         10.0
                    roic         10.0
        capex_to_revenue         10.0


In [11]:
# 결측치 시각화 (Plotly)
missing_top = missing_df[missing_df['missing_pct'] > 0].head(20)

if len(missing_top) > 0:
    fig = px.bar(
        missing_top,
        x='missing_pct',
        y='feature',
        orientation='h',
        title='Feature별 결측치 비율',
        color='missing_pct',
        color_continuous_scale='Reds'
    )
    fig.update_layout(
        yaxis={'categoryorder': 'total ascending'},
        xaxis_title='결측치 비율 (%)',
        yaxis_title='Feature',
        height=500,
        showlegend=False
    )
    fig.show()
else:
    print("✅ 결측치 없음 - 시각화 생략")

## 5. Feature 분포 시각화

In [12]:
# 주요 Feature 선택
key_features = [
    'revenue_cagr_3y', 'operating_margin', 'roic', 
    'rnd_to_revenue', 'fcf_yield', 'debt_to_equity'
]

available_features = [f for f in key_features if f in features_df.columns]

# 히스토그램 (Plotly)
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=available_features
)

for i, feature in enumerate(available_features):
    row = i // 3 + 1
    col = i % 3 + 1
    
    fig.add_trace(
        go.Histogram(
            x=features_df[feature].dropna(),
            name=feature,
            nbinsx=20,
            marker_color='steelblue'
        ),
        row=row, col=col
    )

fig.update_layout(
    title='주요 Feature 분포',
    height=500,
    showlegend=False
)
fig.show()

In [13]:
# Box Plot (Plotly)
fig = go.Figure()

for feature in available_features:
    fig.add_trace(
        go.Box(
            y=features_df[feature].dropna(),
            name=feature,
            boxpoints='outliers'
        )
    )

fig.update_layout(
    title='주요 Feature Box Plot',
    height=500,
    showlegend=False
)
fig.show()

## 6. 이상치 탐지

In [14]:
# IQR 기반 이상치 탐지
def detect_outliers_iqr(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = series[(series < lower) | (series > upper)]
    return len(outliers), lower, upper

outlier_report = []

for col in features_df.columns:
    data = features_df[col].dropna()
    if len(data) > 3:  # 최소 데이터 필요
        n_outliers, lower, upper = detect_outliers_iqr(data)
        outlier_report.append({
            'feature': col,
            'n_outliers': n_outliers,
            'outlier_pct': n_outliers / len(data) * 100,
            'lower_bound': lower,
            'upper_bound': upper
        })

outlier_df = pd.DataFrame(outlier_report).sort_values('outlier_pct', ascending=False)

print("=== 이상치 비율 (상위 10개) ===")
print()
print(outlier_df.head(10).to_string(index=False))

=== 이상치 비율 (상위 10개) ===

               feature  n_outliers  outlier_pct  lower_bound  upper_bound
       revenue_cagr_3y           3    30.000000     0.039875     0.171821
    price_momentum_12m           3    30.000000    -0.351825     0.090513
     interest_coverage           2    22.222222   -86.171875   211.703125
          ev_to_ebitda           2    22.222222     5.396617    44.364056
       fcf_consistency           2    22.222222     0.253398     1.312991
                  roic           2    22.222222    -0.005755     0.462858
             peg_ratio           2    22.222222    -2.498299     7.815968
total_investment_ratio           2    20.000000    -0.046023     0.274745
              pb_ratio           2    20.000000   -18.124154    42.489719
       rnd_growth_rate           1    16.666667    -0.049780     0.340384


In [15]:
# 이상치 비율 시각화 (Plotly)
outlier_top = outlier_df.head(15)

fig = px.bar(   
    outlier_top,
    x='outlier_pct',
    y='feature',
    orientation='h',
    title='Feature별 이상치 비율',
    color='outlier_pct',
    color_continuous_scale='Oranges'
)
fig.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    xaxis_title='이상치 비율 (%)',
    yaxis_title='Feature',
    height=500,
    showlegend=False
)
fig.show()

## 7. Feature 상관관계

In [16]:
# 상관관계 계산
corr_matrix = features_df.corr()

# 상관관계 히트맵 (Plotly)
fig = px.imshow(
    corr_matrix,
    title='Feature 상관관계 히트맵',
    color_continuous_scale='RdBu_r',
    aspect='auto',
    zmin=-1,
    zmax=1
)
fig.update_layout(
    height=800,
    width=900
)
fig.show()

In [17]:
# 높은 상관관계 Feature 쌍 찾기 (|r| > 0.8)
high_corr_pairs = []

for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        corr_val = corr_matrix.iloc[i, j]
        if abs(corr_val) > 0.8:
            high_corr_pairs.append({
                'feature_1': corr_matrix.columns[i],
                'feature_2': corr_matrix.columns[j],
                'correlation': corr_val
            })

high_corr_df = pd.DataFrame(high_corr_pairs).sort_values('correlation', ascending=False)

print("=== 높은 상관관계 (|r| > 0.8) ===")
print()
if len(high_corr_df) > 0:
    print(high_corr_df.to_string(index=False))
else:
    print("높은 상관관계 쌍 없음")

=== 높은 상관관계 (|r| > 0.8) ===

               feature_1                feature_2  correlation
                ps_ratio            ev_to_revenue     0.999892
        operating_margin               net_margin     0.994883
              net_margin               fcf_margin     0.986573
       price_momentum_1m          price_to_sma_50     0.986246
        operating_margin               fcf_margin     0.981014
                pe_ratio             ev_to_ebitda     0.978424
    gross_profit_cagr_3y       net_income_cagr_3y     0.978410
operating_income_cagr_3y       net_income_cagr_3y     0.971410
    gross_profit_cagr_3y operating_income_cagr_3y     0.967382
         revenue_cagr_3y            current_ratio     0.962600
operating_income_cagr_3y   operating_margin_trend     0.962275
         rnd_growth_rate            current_ratio     0.961633
         revenue_cagr_3y     gross_profit_cagr_3y     0.959328
      net_income_cagr_3y   operating_margin_trend     0.956322
         rnd_growth_rate  

## 8. 종목별 Feature 요약

In [18]:
# 종목별 요약 통계
summary_features = ['revenue_cagr_3y', 'operating_margin', 'roic', 
                    'rnd_to_revenue', 'total_investment_ratio', 'fcf_yield']

available_summary = [f for f in summary_features if f in features_df.columns]

print("=== 종목별 주요 Feature ===")
print()
print(features_df[available_summary].round(3).to_string())

=== 종목별 주요 Feature ===

        revenue_cagr_3y  operating_margin   roic  rnd_to_revenue  total_investment_ratio  fcf_yield
ticker                                                                                             
AAPL              0.018             0.320  0.823           0.083                   0.114      0.024
MSFT              0.124             0.456  0.283           0.115                   0.344      0.020
NVDA              0.693             0.624  0.872           0.099                   0.124      0.013
GOOGL             0.108             0.321  0.287           0.141                   0.291      0.019
AMZN              0.107             0.108  0.175             NaN                   0.130      0.013
TSLA              0.220             0.079  0.088           0.046                   0.163      0.002
UNH               0.117             0.081  0.170             NaN                   0.009      0.069
JPM               0.117               NaN    NaN             NaN            

In [19]:
# Baillie Gifford 스타일 랭킹 (R&D 집중도 + 성장성)
if 'total_investment_ratio' in features_df.columns and 'revenue_cagr_3y' in features_df.columns:
    ranking_df = features_df[['total_investment_ratio', 'revenue_cagr_3y']].copy()
    ranking_df['bg_score'] = (
        ranking_df['total_investment_ratio'].rank(pct=True) * 0.5 +
        ranking_df['revenue_cagr_3y'].rank(pct=True) * 0.5
    )
    ranking_df = ranking_df.sort_values('bg_score', ascending=False)
    
    print("=== Baillie Gifford 스타일 랭킹 ===")
    print("(R&D+CapEx 집중도 50% + 매출 성장률 50%)")
    print()
    print(ranking_df.round(3).to_string())

=== Baillie Gifford 스타일 랭킹 ===
(R&D+CapEx 집중도 50% + 매출 성장률 50%)

        total_investment_ratio  revenue_cagr_3y  bg_score
ticker                                                   
MSFT                     0.344            0.124      0.90
TSLA                     0.163            0.220      0.85
NVDA                     0.124            0.693      0.80
GOOGL                    0.291            0.108      0.70
AMZN                     0.130            0.107      0.55
UNH                      0.009            0.117      0.40
JPM                      0.000            0.117      0.40
CAT                      0.082            0.083      0.35
AAPL                     0.114            0.018      0.30
XOM                      0.072            0.070      0.25


In [20]:
# BG Score 시각화 (Plotly)
if 'bg_score' in ranking_df.columns:
    fig = px.bar(
        ranking_df.reset_index(),
        x='bg_score',
        y='ticker',
        orientation='h',
        title='Baillie Gifford 스타일 랭킹',
        color='bg_score',
        color_continuous_scale='Greens'
    )
    fig.update_layout(
        yaxis={'categoryorder': 'total ascending'},
        xaxis_title='BG Score',
        yaxis_title='',  # 빈 문자열로 변경
        height=400,
        showlegend=False
    )
    fig.show()

## 9. 결론 및 다음 단계

In [21]:
print("=" * 50)
print("📋 Feature Validation 결론 (10개 종목 테스트)")
print("=" * 50)

print(f"\n✅ 총 Feature 수: {len(features_df.columns)}개")
print(f"✅ 테스트 종목 수: {len(features_df)}개")

# 결측치 요약
high_missing = missing_df[missing_df['missing_pct'] > 50]['feature'].tolist()
print(f"\n⚠️ 결측치 50% 이상 Feature: {len(high_missing)}개")
if high_missing:
    print(f"   → {high_missing}")

# 상관관계 요약
print(f"⚠️ 높은 상관관계 쌍 (|r|>0.8): {len(high_corr_df)}개 → Feature 선택 필요")

# BG 스타일 Top 3
print("\n📊 Baillie Gifford 스타일 Top 3")
print(ranking_df.head(3)[['bg_score']].to_string())

# 다음 단계
print("\n" + "=" * 50)
print("🔜 다음 단계")
print("=" * 50)
print("""
※ 이 노트북은 10개 종목 테스트용입니다.
  전체 데이터 분석은 03_feature_engineering.ipynb에서 수행합니다.

02_data_collection.ipynb
   - S&P 500 전체 (502개) 데이터 수집
   - 종목별 데이터 시작 연도 저장

03_feature_engineering.ipynb
   - 전체 데이터 기준 Feature Selection (42개 → 26개)
   - 결측치 50%+ 제거: rnd_growth_rate, rnd_to_revenue
   - 상관관계 0.8+ 중복 제거: ~14개
   - 전처리: 중앙값 대체, Winsorization (1%), StandardScaler

04_model_training.ipynb
   - Target: 5년 5배 달성 여부
   - 모델: XGBoost + Time Series CV
""")

📋 Feature Validation 결론 (10개 종목 테스트)

✅ 총 Feature 수: 42개
✅ 테스트 종목 수: 10개

⚠️ 결측치 50% 이상 Feature: 1개
   → ['revenue_cagr_5y']
⚠️ 높은 상관관계 쌍 (|r|>0.8): 78개 → Feature 선택 필요

📊 Baillie Gifford 스타일 Top 3
        bg_score
ticker          
MSFT        0.90
TSLA        0.85
NVDA        0.80

🔜 다음 단계

※ 이 노트북은 10개 종목 테스트용입니다.
  전체 데이터 분석은 03_feature_engineering.ipynb에서 수행합니다.

02_data_collection.ipynb
   - S&P 500 전체 (502개) 데이터 수집
   - 종목별 데이터 시작 연도 저장

03_feature_engineering.ipynb
   - 전체 데이터 기준 Feature Selection (42개 → 26개)
   - 결측치 50%+ 제거: rnd_growth_rate, rnd_to_revenue
   - 상관관계 0.8+ 중복 제거: ~14개
   - 전처리: 중앙값 대체, Winsorization (1%), StandardScaler

04_model_training.ipynb
   - Target: 5년 5배 달성 여부
   - 모델: XGBoost + Time Series CV

